In [1]:
#1. Data Ingestion and Preparation

from langchain_community.document_loaders import PyPDFLoader


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [4]:
import os
os.getcwd()

'c:\\Users\\Madhu Kumar\\Desktop\\My Learnings\\LLomps\\Document_Portal\\notebooks'

In [5]:
file_path=os.path.join(os.getcwd(), "data", "sample.pdf")

In [7]:
loader=PyPDFLoader(file_path)
documents=loader.load()
len(documents)

21

In [8]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
    length_function=len
)

In [9]:
docs=text_splitter.split_documents(documents)
len(docs)

316

In [10]:
docs[0].metadata

{'producer': 'pdfTeX-1.40.25',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2024-03-28T00:54:45+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2024-03-28T00:54:45+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
 'subject': '',
 'title': '',
 'trapped': '/False',
 'source': 'c:\\Users\\Madhu Kumar\\Desktop\\My Learnings\\LLomps\\Document_Portal\\notebooks\\data\\sample.pdf',
 'total_pages': 21,
 'page': 0,
 'page_label': '1'}

In [14]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
model=ChatGroq(model="qwen/qwen3-32b")
model.invoke("what is the capital of Andhra Pradesh?").content


"<think>\nOkay, so I need to figure out what the capital of Andhra Pradesh is. Let me start by recalling what I know about Indian states. Andhra Pradesh is a state in India, right? I think it's in the southern part. Now, capitals of Indian states... Some I know, like Tamil Nadu's capital is Chennai, Kerala's is Thiruvananthapuram. But Andhra Pradesh... Hmm.\n\nWait, I remember that Andhra Pradesh used to have Hyderabad as its capital, but I think there was a change. Oh, right! There was a new capital created after the bifurcation of the old Andhra Pradesh. The old state was split into Telangana and the new Andhra Pradesh. So the old capital, Hyderabad, went to Telangana. Then Andhra Pradesh needed a new capital. I think they built a new city for it. What was the name again? Amravati? Or maybe Amaravati? I'm a bit confused between the two. \n\nWait, Amravati is a city in the new Andhra Pradesh, and it's supposed to be the capital. But I also remember that there was some talk about shift

In [21]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


In [22]:
vectorstore=FAISS.from_documents(docs, embedding_model)

###   Retrieval



In [29]:
relevant_doc=vectorstore.similarity_search("rag techniques.")
print(relevant_doc[0].page_content)

leading to rapid development in RAG studies. As research
progressed, the enhancement of RAG was no longer limited
to the inference stage but began to incorporate more with LLM
fine-tuning techniques.
The burgeoning field of RAG has experienced swift growth,
yet it has not been accompanied by a systematic synthesis that
could clarify its broader trajectory. This survey endeavors to
fill this gap by mapping out the RAG process and charting


In [28]:
print(relevant_doc[1].page_content)

to generate a well-informed answer.
The RAG research paradigm is continuously evolving, and
we categorize it into three stages: Naive RAG, Advanced
RAG, and Modular RAG, as showed in Figure 3. Despite
RAG method are cost-effective and surpass the performance
of the native LLM, they also exhibit several limitations.
The development of Advanced RAG and Modular RAG is
a response to these specific shortcomings in Naive RAG.
A. Naive RAG
The Naive RAG research paradigm represents the earli-


In [33]:
relevant_doc=vectorstore.similarity_search("rag techniques.",k=3)
print(relevant_doc)

[Document(id='920b646a-a042-4c1d-8f8c-9049a369103f', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-03-28T00:54:45+00:00', 'author': '', 'keywords': '', 'moddate': '2024-03-28T00:54:45+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'c:\\Users\\Madhu Kumar\\Desktop\\My Learnings\\LLomps\\Document_Portal\\notebooks\\data\\sample.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1'}, page_content='leading to rapid development in RAG studies. As research\nprogressed, the enhancement of RAG was no longer limited\nto the inference stage but began to incorporate more with LLM\nfine-tuning techniques.\nThe burgeoning field of RAG has experienced swift growth,\nyet it has not been accompanied by a systematic synthesis that\ncould clarify its broader trajectory. This survey endeavors to\nfill this gap by mapping out the

In [62]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [63]:
retriever.invoke("hybrid rag techniques?")

[Document(id='920b646a-a042-4c1d-8f8c-9049a369103f', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-03-28T00:54:45+00:00', 'author': '', 'keywords': '', 'moddate': '2024-03-28T00:54:45+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'c:\\Users\\Madhu Kumar\\Desktop\\My Learnings\\LLomps\\Document_Portal\\notebooks\\data\\sample.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1'}, page_content='leading to rapid development in RAG studies. As research\nprogressed, the enhancement of RAG was no longer limited\nto the inference stage but began to incorporate more with LLM\nfine-tuning techniques.\nThe burgeoning field of RAG has experienced swift growth,\nyet it has not been accompanied by a systematic synthesis that\ncould clarify its broader trajectory. This survey endeavors to\nfill this gap by mapping out the

Question: user question


Context: based on the question retrieving the info from the vector database

In [75]:
prompt_template = """
        Answer the question based on the context provided below. 
        If the context does not contain sufficient information, respond with: 
        "I do not have enough information about this."

        Context: {context}

        Question: {question}

        Answer:"""

In [76]:
from langchain_core.prompts import PromptTemplate


In [77]:
prompt=PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [78]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n        Answer the question based on the context provided below. \n        If the context does not contain sufficient information, respond with: \n        "I do not have enough information about this."\n\n        Context: {context}\n\n        Question: {question}\n\n        Answer:')

In [79]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

In [80]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [81]:
from langchain_core.runnables import RunnablePassthrough

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | model
)


In [82]:
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000014D34EDE5C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000014D34EDEB90>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [83]:
rag_chain.invoke("what is RAG?")

AIMessage(content='<think>\nOkay, so the user is asking, "What is RAG?" Let me look through the provided context to find the answer.\n\nFirst, I see that the documents mention RAG research paradigms categorized into three stages: Naive RAG, Advanced RAG, and Modular RAG. The context talks about how RAG methods are cost-effective and perform better than native LLMs but have limitations. It also mentions that Advanced and Modular RAG address the shortcomings of Naive RAG.\n\nAnother part of the context refers to the tripartite foundation of RAG frameworks: retrieval, generation, and augmentation techniques. The documents discuss the progression of RAG systems, including enhancements with LLM fine-tuning and evaluation methods covering tasks, datasets, metrics, and benchmarks.\n\nPutting this together, RAG stands for Retrieval-Augmented Generation. It\'s a method that combines retrieval of information from external sources with the generation capabilities of language models to enhance the